# TensorLogic Python Tutorial: Getting Started

This notebook provides a comprehensive introduction to TensorLogic's Python API.

TensorLogic is a **logic-as-tensor planning layer** that compiles logical expressions into tensor computations. It enables neural/symbolic/probabilistic reasoning within a unified tensor computation framework.

## What You'll Learn

1. Basic logical expressions (predicates, AND, OR, NOT)
2. Compilation and execution
3. Compilation strategies (soft, hard, fuzzy, probabilistic)
4. Quantifiers (EXISTS, FORALL)
5. Arithmetic and comparison operations
6. Complex nested expressions
7. Adapter types for advanced usage

## Installation

```bash
# From source (requires Rust toolchain)
cd crates/pytensorlogic
maturin develop
```

## Setup

In [ ]:
import pytensorlogic as tl
import numpy as np
import matplotlib.pyplot as plt

print(f"TensorLogic version: {tl.__version__}")

## 1. Basic Logical Expressions

### 1.1 Terms (Variables and Constants)

Terms are the building blocks of logical expressions:

In [ ]:
# Create a variable
x = tl.var("x")
print(f"Variable: {x.name()}, is_var: {x.is_var()}, is_const: {x.is_const()}")

# Create a constant
alice = tl.const("alice")
print(f"Constant: {alice.name()}, is_var: {alice.is_var()}, is_const: {alice.is_const()}")

### 1.2 Predicates

Predicates represent properties or relations:

In [ ]:
# Unary predicate: P(x)
x = tl.var("x")
p = tl.pred("P", [x])

# Binary predicate: knows(x, y)
y = tl.var("y")
knows = tl.pred("knows", [x, y])

print("Created predicates P(x) and knows(x, y)")

### 1.3 Logical Connectives

Combine expressions using AND, OR, NOT:

In [ ]:
x = tl.var("x")
p = tl.pred("P", [x])
q = tl.pred("Q", [x])

# Conjunction: P(x) AND Q(x)
p_and_q = tl.and_expr(p, q)

# Disjunction: P(x) OR Q(x)
p_or_q = tl.or_expr(p, q)

# Negation: NOT P(x)
not_p = tl.not_expr(p)

print("Created logical connectives")

## 2. Compilation and Execution

### 2.1 Compile Expressions to Tensor Graphs

TensorLogic compiles logical expressions into einsum graphs:

In [ ]:
# Create a simple AND expression
x = tl.var("x")
p = tl.pred("P", [x])
q = tl.pred("Q", [x])
expr = tl.and_expr(p, q)

# Compile to tensor graph
graph = tl.compile(expr)

# Inspect graph statistics
stats = graph.stats()
print(f"Graph statistics: {stats}")

### 2.2 Execute Graphs with Input Data

Execute compiled graphs with numpy arrays:

In [ ]:
# Prepare input data (3 domain elements)
p_data = np.array([1.0, 0.8, 0.2])  # P(x) probabilities
q_data = np.array([0.9, 0.7, 0.1])  # Q(x) probabilities

# Execute the graph
result = tl.execute(graph, {"P": p_data, "Q": q_data})

print(f"Input P: {p_data}")
print(f"Input Q: {q_data}")
print(f"Result (P AND Q): {result}")
print(f"\nVerify (product): {p_data * q_data}")

### 2.3 Visualize Results

In [ ]:
# Visualize AND operation
x_labels = ['Element 0', 'Element 1', 'Element 2']
x_pos = np.arange(len(x_labels))

fig, ax = plt.subplots(figsize=(10, 6))
width = 0.25

ax.bar(x_pos - width, p_data, width, label='P(x)', color='blue', alpha=0.7)
ax.bar(x_pos, q_data, width, label='Q(x)', color='green', alpha=0.7)
ax.bar(x_pos + width, result, width, label='P(x) AND Q(x)', color='red', alpha=0.7)

ax.set_xlabel('Domain Elements')
ax.set_ylabel('Probability')
ax.set_title('TensorLogic AND Operation (Product Strategy)')
ax.set_xticks(x_pos)
ax.set_xticklabels(x_labels)
ax.legend()
ax.grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.show()

## 3. Compilation Strategies

TensorLogic supports multiple strategies for mapping logical operations to tensor operations:

- **soft_differentiable**: Default, product AND, smooth operations
- **hard_boolean**: Min/Max operations for crisp logic
- **fuzzy_godel**: Gödel t-norm (min AND, max OR)
- **fuzzy_product**: Product t-norm
- **fuzzy_lukasiewicz**: Łukasiewicz t-norm
- **probabilistic**: Probabilistic interpretation

### 3.1 Compare Strategies

In [ ]:
# Test data
p_data = np.array([0.3, 0.5, 0.7, 0.9])
q_data = np.array([0.4, 0.6, 0.5, 0.8])

# Create expression
x = tl.var("x")
p = tl.pred("P", [x])
q = tl.pred("Q", [x])
expr = tl.and_expr(p, q)

# Compile with different strategies
strategies = [
    ("Soft Differentiable", tl.CompilationConfig()),
    ("Hard Boolean", tl.CompilationConfig()),
    ("Fuzzy Gödel", tl.CompilationConfig()),
]

results = {}
for name, config in strategies:
    graph = tl.compile_with_config(expr, config)
    result = tl.execute(graph, {"P": p_data, "Q": q_data})
    results[name] = result
    print(f"{name:20} -> {result}")

### 3.2 Visualize Strategy Differences

In [ ]:
fig, ax = plt.subplots(figsize=(12, 6))
x_pos = np.arange(len(p_data))
width = 0.15

# Plot inputs
ax.bar(x_pos - 2*width, p_data, width, label='P(x)', color='blue', alpha=0.6)
ax.bar(x_pos - width, q_data, width, label='Q(x)', color='green', alpha=0.6)

# Plot strategy results
colors = ['red', 'orange', 'purple']
for i, (name, result) in enumerate(results.items()):
    ax.bar(x_pos + i*width, result, width, label=name, color=colors[i], alpha=0.7)

ax.set_xlabel('Domain Elements')
ax.set_ylabel('Probability')
ax.set_title('AND Operation Across Different Compilation Strategies')
ax.set_xticks(x_pos)
ax.set_xticklabels([f'x{i}' for i in range(len(p_data))])
ax.legend()
ax.grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.show()

## 4. Quantifiers

### 4.1 Existential Quantification (EXISTS)

EXISTS reduces over a variable dimension:

In [ ]:
# EXISTS x in Domain. P(x)
x = tl.var("x")
p = tl.pred("P", [x])
exists_expr = tl.exists("x", "Domain", p)

# Compile and execute
graph = tl.compile(exists_expr)
p_data = np.array([0.2, 0.3, 0.8, 0.1])  # One element has high probability

result = tl.execute(graph, {"P": p_data})
print(f"P(x) values: {p_data}")
print(f"EXISTS x. P(x): {result}")
print(f"\nInterpretation: At least one element satisfies P with high probability")

### 4.2 Universal Quantification (FORALL)

FORALL also reduces over a variable dimension:

In [ ]:
# FORALL x in Domain. P(x)
x = tl.var("x")
p = tl.pred("P", [x])
forall_expr = tl.forall("x", "Domain", p)

# Compile and execute
graph = tl.compile(forall_expr)

# Test with different scenarios
scenarios = [
    ("All high", np.array([0.9, 0.95, 0.92, 0.88])),
    ("Mixed", np.array([0.9, 0.5, 0.8, 0.7])),
    ("All low", np.array([0.1, 0.2, 0.15, 0.3])),
]

for name, p_data in scenarios:
    result = tl.execute(graph, {"P": p_data})
    print(f"{name:12} P(x): {p_data} -> FORALL: {result[0]:.3f}")

## 5. Arithmetic Operations

TensorLogic supports arithmetic operations on logical expressions:

In [ ]:
x = tl.var("x")
p = tl.pred("P", [x])
q = tl.pred("Q", [x])

# Arithmetic operations
add_expr = tl.add(p, q)          # P(x) + Q(x)
sub_expr = tl.sub(p, q)          # P(x) - Q(x)
mul_expr = tl.mul(p, q)          # P(x) * Q(x)
div_expr = tl.div(p, tl.constant(2.0))  # P(x) / 2.0

# Test data
p_data = np.array([1.0, 2.0, 3.0])
q_data = np.array([0.5, 1.0, 1.5])

# Execute each operation
operations = [
    ("Addition", add_expr, {"P": p_data, "Q": q_data}),
    ("Subtraction", sub_expr, {"P": p_data, "Q": q_data}),
    ("Multiplication", mul_expr, {"P": p_data, "Q": q_data}),
    ("Division", div_expr, {"P": p_data}),
]

for name, expr, inputs in operations:
    graph = tl.compile(expr)
    result = tl.execute(graph, inputs)
    print(f"{name:15} -> {result}")

## 6. Comparison Operations

Compare expressions using relational operators:

In [ ]:
x = tl.var("x")
p = tl.pred("P", [x])
q = tl.pred("Q", [x])

# Comparison operations
eq_expr = tl.eq(p, q)    # P(x) == Q(x)
lt_expr = tl.lt(p, q)    # P(x) < Q(x)
gt_expr = tl.gt(p, q)    # P(x) > Q(x)
lte_expr = tl.lte(p, q)  # P(x) <= Q(x)
gte_expr = tl.gte(p, q)  # P(x) >= Q(x)

# Test data
p_data = np.array([1.0, 2.0, 3.0, 2.0])
q_data = np.array([1.0, 3.0, 2.0, 2.0])

# Execute comparisons
comparisons = [
    ("Equal", eq_expr),
    ("Less Than", lt_expr),
    ("Greater Than", gt_expr),
    ("Less or Equal", lte_expr),
    ("Greater or Equal", gte_expr),
]

print(f"P(x): {p_data}")
print(f"Q(x): {q_data}")
print()

for name, expr in comparisons:
    graph = tl.compile(expr)
    result = tl.execute(graph, {"P": p_data, "Q": q_data})
    print(f"{name:18} -> {result}")

## 7. Conditional Expressions

IF-THEN-ELSE for conditional logic:

In [ ]:
x = tl.var("x")
condition = tl.pred("C", [x])    # Condition
then_expr = tl.pred("T", [x])    # Then branch
else_expr = tl.pred("E", [x])    # Else branch

# IF C(x) THEN T(x) ELSE E(x)
ite_expr = tl.if_then_else(condition, then_expr, else_expr)

# Test data
c_data = np.array([1.0, 1.0, 0.0, 0.0])  # Condition: true, true, false, false
t_data = np.array([0.8, 0.9, 0.7, 0.6])  # Then values
e_data = np.array([0.2, 0.3, 0.4, 0.5])  # Else values

graph = tl.compile(ite_expr)
result = tl.execute(graph, {"C": c_data, "T": t_data, "E": e_data})

print(f"Condition: {c_data}")
print(f"Then:      {t_data}")
print(f"Else:      {e_data}")
print(f"Result:    {result}")
print(f"\nExpected: [0.8, 0.9, 0.4, 0.5] (picks T when C=1, E when C=0)")

## 8. Complex Nested Expressions

### 8.1 De Morgan's Law

Verify: NOT(P AND Q) = (NOT P) OR (NOT Q)

In [ ]:
x = tl.var("x")
p = tl.pred("P", [x])
q = tl.pred("Q", [x])

# Left side: NOT(P AND Q)
left = tl.not_expr(tl.and_expr(p, q))

# Right side: (NOT P) OR (NOT Q)
right = tl.or_expr(tl.not_expr(p), tl.not_expr(q))

# Test data
p_data = np.array([0.0, 0.0, 1.0, 1.0])
q_data = np.array([0.0, 1.0, 0.0, 1.0])

# Execute both sides
left_graph = tl.compile(left)
right_graph = tl.compile(right)

left_result = tl.execute(left_graph, {"P": p_data, "Q": q_data})
right_result = tl.execute(right_graph, {"P": p_data, "Q": q_data})

print(f"P:                    {p_data}")
print(f"Q:                    {q_data}")
print(f"NOT(P AND Q):         {left_result}")
print(f"(NOT P) OR (NOT Q):   {right_result}")
print(f"\nDifference: {np.abs(left_result - right_result)}")
print(f"De Morgan's law holds: {np.allclose(left_result, right_result)}")

### 8.2 Implication

P → Q (P implies Q)

In [ ]:
x = tl.var("x")
p = tl.pred("P", [x])
q = tl.pred("Q", [x])

# P → Q (implication)
impl_expr = tl.imply(p, q)

# Truth table for implication
p_data = np.array([0.0, 0.0, 1.0, 1.0])
q_data = np.array([0.0, 1.0, 0.0, 1.0])

graph = tl.compile(impl_expr)
result = tl.execute(graph, {"P": p_data, "Q": q_data})

print("Truth table for P → Q:")
print("P    Q    P→Q")
print("-" * 20)
for i in range(len(p_data)):
    print(f"{p_data[i]:.1f}  {q_data[i]:.1f}  {result[i]:.3f}")

print("\nImplication is false only when P is true and Q is false")

## 9. Adapter Types (Advanced)

### 9.1 Domain Information

Define domain metadata:

In [ ]:
# Create domain information
person_domain = tl.domain_info("Person", 10)  # Domain with 10 elements
location_domain = tl.domain_info("Location", 5)  # Domain with 5 elements

print(f"Created domain 'Person' with cardinality 10")
print(f"Created domain 'Location' with cardinality 5")

### 9.2 Predicate Information

Define predicate signatures:

In [ ]:
# Create predicate information
# isHappy: Person -> Bool
is_happy = tl.predicate_info("isHappy", ["Person"])

# knows: Person × Person -> Bool
knows = tl.predicate_info("knows", ["Person", "Person"])

# at: Person × Location -> Bool
at_location = tl.predicate_info("at", ["Person", "Location"])

print("Created predicate signatures:")
print("- isHappy(Person)")
print("- knows(Person, Person)")
print("- at(Person, Location)")

### 9.3 Symbol Table

Manage domains and predicates:

In [ ]:
# Create symbol table
sym_table = tl.symbol_table()

# Symbol tables are used internally by the compiler
# to track variable bindings, domain information, and predicate signatures

print("Created symbol table for managing compilation context")

### 9.4 Compiler Context

Control compilation behavior:

In [ ]:
# Create compiler context
ctx = tl.compiler_context()

# Compiler contexts are used for advanced compilation control
# such as strategy selection, optimization passes, and debugging

print("Created compiler context for advanced compilation control")

## 10. Practical Example: Social Network Reasoning

Let's model a simple social network scenario:
- People can be friends
- Happy people make their friends happy (with some probability)
- Query: Given initial happiness, what's the happiness distribution?

In [ ]:
# Define the rule:
# FORALL x. (happy(x) AND EXISTS y. (friends(x,y) AND happy(y))) -> happy(x)
#
# Simplified version: If you have a happy friend, you become happier

x = tl.var("x")
y = tl.var("y")

# Predicates
happy_x = tl.pred("happy", [x])
happy_y = tl.pred("happy", [y])
friends_xy = tl.pred("friends", [x, y])

# Rule: happy(y) AND friends(x,y)
happy_friend = tl.and_expr(happy_y, friends_xy)

# EXISTS y. (happy(y) AND friends(x,y))
has_happy_friend = tl.exists("y", "Person", happy_friend)

# Compile the query
graph = tl.compile(has_happy_friend)

# Social network: 4 people (0, 1, 2, 3)
# Initial happiness: person 0 is very happy
happy_data = np.array([0.9, 0.2, 0.3, 0.1])

# Friendship matrix (4x4): rows are observers (x), cols are friends (y)
# Person 0 is friends with everyone
# Person 1 is friends with 0 and 2
# Person 2 is friends with 1
# Person 3 is friends with 0
friends_data = np.array([
    [0.0, 1.0, 1.0, 1.0],  # Person 0's friendships
    [1.0, 0.0, 1.0, 0.0],  # Person 1's friendships
    [0.0, 1.0, 0.0, 0.0],  # Person 2's friendships
    [1.0, 0.0, 0.0, 0.0],  # Person 3's friendships
])

# Execute: for each person, do they have a happy friend?
result = tl.execute(graph, {"happy": happy_data, "friends": friends_data})

print("Social Network Analysis:")
print("="*40)
print(f"Initial happiness: {happy_data}")
print(f"\nHas happy friend?  {result}")
print("\nInterpretation:")
for i in range(len(result)):
    friends = np.where(friends_data[i] > 0)[0]
    print(f"Person {i}: happiness={happy_data[i]:.1f}, "
          f"friends={list(friends)}, "
          f"has_happy_friend={result[i]:.3f}")

### Visualize Social Network

In [ ]:
import matplotlib.patches as patches

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))

# Left plot: Network structure
positions = np.array([[0, 1], [1, 1], [0, 0], [1, 0]])  # 2D positions

# Draw nodes
for i, (pos, happiness) in enumerate(zip(positions, happy_data)):
    circle = patches.Circle(pos, 0.15, facecolor=plt.cm.RdYlGn(happiness), 
                           edgecolor='black', linewidth=2)
    ax1.add_patch(circle)
    ax1.text(pos[0], pos[1], str(i), ha='center', va='center', 
            fontsize=16, fontweight='bold')

# Draw edges
for i in range(len(friends_data)):
    for j in range(i+1, len(friends_data)):
        if friends_data[i, j] > 0:
            ax1.plot([positions[i, 0], positions[j, 0]], 
                    [positions[i, 1], positions[j, 1]], 
                    'k-', alpha=0.3, linewidth=2)

ax1.set_xlim(-0.5, 1.5)
ax1.set_ylim(-0.5, 1.5)
ax1.set_aspect('equal')
ax1.set_title('Social Network\n(Node color = happiness)', fontsize=14)
ax1.axis('off')

# Right plot: Happiness analysis
x_pos = np.arange(len(happy_data))
width = 0.35

ax2.bar(x_pos - width/2, happy_data, width, label='Initial Happiness', 
       color='blue', alpha=0.7)
ax2.bar(x_pos + width/2, result, width, label='Has Happy Friend', 
       color='green', alpha=0.7)

ax2.set_xlabel('Person ID', fontsize=12)
ax2.set_ylabel('Probability', fontsize=12)
ax2.set_title('Happiness Distribution', fontsize=14)
ax2.set_xticks(x_pos)
ax2.set_xticklabels([f'Person {i}' for i in range(len(happy_data))])
ax2.legend()
ax2.grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.show()

## 11. Summary and Next Steps

### What You've Learned

✅ Creating logical expressions with predicates, variables, and connectives  
✅ Compiling expressions into tensor graphs  
✅ Executing graphs with numpy arrays  
✅ Using different compilation strategies (soft, hard, fuzzy)  
✅ Working with quantifiers (EXISTS, FORALL)  
✅ Arithmetic and comparison operations  
✅ Complex nested expressions and logical laws  
✅ Practical reasoning tasks (social network example)  

### Next Steps

1. **Integration with Neural Networks**: Use TensorLogic as a planning layer for neural-symbolic AI
2. **Training**: Explore training scenarios with gradient-based optimization
3. **Advanced Adapters**: Deep dive into symbol tables and compiler contexts
4. **Performance**: Benchmark different strategies and graph optimizations
5. **Real-world Applications**: Knowledge graphs, constraint satisfaction, probabilistic reasoning

### Resources

- **GitHub**: https://github.com/cool-japan/tensorlogic
- **Documentation**: See README.md and CLAUDE.md in repository
- **Paper**: TensorLogic arxiv paper
- **Examples**: Check `examples/` directory for more use cases

### Questions?

Check the issue tracker or explore the test suite in `tests/` for more examples!